In [7]:
import pandas as pd
import numpy as np
import requests
import json
import multiprocessing
import time

text_analytics_base_url = "https://westus.api.cognitive.microsoft.com/text/analytics/v2.0/"
key_phrase_api_url = text_analytics_base_url + "keyPhrases"

print(key_phrase_api_url)

subscription_key = 'COGNITIVE_SERVICES_KEY'
assert subscription_key

responsibilities = pd.read_csv('./responsibilities.csv')
responsibilities


https://westus.api.cognitive.microsoft.com/text/analytics/v2.0/keyPhrases


,candidateID,desc
0,39629_102,"Scheduling coordinator, reschedule appointment..."
1,41201_223,Billing clients Online research for upcoming c...
2,41201_745,ï¿½ Prepare orders to customers specifications...
3,41614_672,working with store manager and associates to i...
4,41753_315,"Setting up and tearing down events, from light..."
5,41753_212,My duties are to help disabled individuals wit...
6,42313_499,Assisting elderly with daily living. Dietary Aide
7,42394_481,Making drive thru times under a minute Making ...
8,42394_460,Set prices on shelves for various products
9,44170_254,( Assist students with physical activ...


In [8]:
chunk_size = 1000
chunks = round(len(responsibilities)/chunk_size)+1
results = []

def processData(chunk, session):
    sampledoc = pd.DataFrame()
    sampledoc['text'] = chunk['desc']
    sampledoc['language'] = 'en'
    sampledoc['id'] = chunk['candidateID']
    col_order = ['id','language','text']
    sampledoc = sampledoc[col_order]
    documents = sampledoc
    documents = {'documents' : eval(documents.to_json(orient='records', force_ascii=False)) }
    headers = {"Ocp-Apim-Subscription-Key": subscription_key}
    response = session.post(key_phrase_api_url, headers=headers, json=documents)
    key_phrases = response.json()
    return pd.DataFrame(key_phrases['documents']).values.tolist()


def collect_results(result):
    results.extend(result)
    


In [9]:
#main code
start_time = time.time()
# Parallel processing of Cognitive Services calls takes ~ 5mins
pool = multiprocessing.Pool(1)#processes=multiprocessing.cpu_count())

#Process in parallel if necessary
if(chunks > 1) :
    for chunk in (np.array_split(responsibilities,chunks)) :
        session = requests.Session()
        print(chunk)
        pool.apply_async(processData, args=(chunk,session, ), callback=collect_results)
    print("closing pool")
    pool.close()
    pool.join()
else :
    results = processData(responsibilities, requests.Session())
    

# Converts list back to a data frame
df = pd.DataFrame(results)
print("--- %s seconds ---" % (time.time() - start_time))
df

--- 0.2185380458831787 seconds ---


,0,1
0,39629_102,"[qualified appointments, Outbound calls, Sched..."
1,41201_223,"[upcoming cases, Accusatory Instruments, Court..."
2,41201_745,"[customers specifications ï, work areas, diff..."
3,41614_672,"[internal shrink, action, associates, safety a..."
4,41753_315,"[events, Niagara Center, arts center, Shea's, ..."
5,41753_212,"[cooking, everyday needs, disabled individuals..."
6,42313_499,"[elderly, daily living, Dietary Aide]"
7,42394_481,"[minute, end, employees, times, drawers, day, ..."
8,42394_460,"[prices, shelves, various products]"
9,44170_254,"[physical activity, sports, dodgeball, basketb..."
